## GAD-7 DETECTOR
Here I will use the exact same model and parameters as used in the PHQ-8 model. There is no transfer learning.

In [1]:
from data_Loader import dataCreator, dataLoaderPickle, justDatasetLoaderPickle, anxietyDataCreator, dataSaver
from networks import FeaturesModel
from networks_v2 import AttentionModel
from optimizer import train2, test2, optimizerNet
from optimizer import IterMeter

import os
import torch
import torch.nn as nn
from comet_ml import Experiment
import numpy as np

from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split, SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold


c:\Users\ericq\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_loader = "C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\Raw_Data_v2\\RADAR-MDD-KCL-s1\\RADAR-MDD-KCL-s1"
test_loader = ""

name =[]
data = []
audio = []

name = [x[0] for x in os.walk(train_loader)]

name.pop(0)


'C:\\Users\\ericq\\OneDrive\\Escriptori\\TELECOM\\MSC MATT\\TFM\\The research question\\The Solution\\CNN+RNN\\Raw_Data_v2\\RADAR-MDD-KCL-s1\\RADAR-MDD-KCL-s1'

In [3]:
train_data, test_data, dataset = anxietyDataCreator(train_loader, "GAD7", "Scripted_Eng_GAD7")

Signal processed 1_1
New Spectrogram
The new spectrogram has a target
Label: 0	ID: 00d1d60a-15cf-481b-8264-91705b6f8d97	Original label: 2.0
Label: 0	ID: 00d1d60a-15cf-481b-8264-91705b6f8d97	Original label: 2.0
(2, 500, 40) Spectrogram shape
Length Spectrograms:  (3, 500, 40)
Length labels:  2
Signal processed 2_1
New Spectrogram
The new spectrogram has a target
Label: 0	ID: 00d1d60a-15cf-481b-8264-91705b6f8d97	Original label: 2.0
Label: 0	ID: 00d1d60a-15cf-481b-8264-91705b6f8d97	Original label: 2.0
(2, 500, 40) Spectrogram shape
Length Spectrograms:  (5, 500, 40)
Length labels:  4
Signal processed 3_1
New Spectrogram
The new spectrogram has a target
Label: 1	ID: 00d1d60a-15cf-481b-8264-91705b6f8d97	Original label: 5.0
Label: 1	ID: 00d1d60a-15cf-481b-8264-91705b6f8d97	Original label: 5.0
(2, 500, 40) Spectrogram shape
Length Spectrograms:  (7, 500, 40)
Length labels:  6
Signal processed 4_1
New Spectrogram
The new spectrogram has a target
Label: 1	ID: 00d1d60a-15cf-481b-8264-91705b6f8d9

### DataLoader and K-FOLD

In [6]:
dataset = justDatasetLoaderPickle("Scripted_Eng_GAD7")

In [8]:
#Constants
learning_Rate = 0.0005
batch_size = 128
epochs = 50
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}
experiment = Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 6,
    "n_rnn_layers": 1,
    "rnn_dim": 256,
    "h_rnn_layers": 128,
    "n_class": 4,
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


In [9]:
#Building and training Model
model = AttentionModel(
    hparams['n_cnn_layers'],
    hparams['rnn_dim'],
    hparams['h_rnn_layers'],
    hparams['n_rnn_layers'],
    hparams['n_class'],
    hparams['stride'],
    hparams['dropout']
).to(device).float()

print(model)
print('Number Model Parameters', sum([param.nelement() for param in model.parameters()]))

#Optimizing Model
optimizer, scheduler = optimizerNet(model, hparams)
weights = [1.0,1.0,1.0,0.85]
class_weights = torch.FloatTensor(weights)
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
iter_meter = IterMeter()

AttentionModel(
  (dense): Conv1d(500, 32, kernel_size=(1,), stride=(1,))
  (cnn): Sequential(
    (0): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (2): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_runn

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform_(weight, -stdv, stdv)


In [10]:
history = {'Train loss':[], 'Test Loss':[], 'Train Accuracy':[], 'Test Accuracy':[]}

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    print('Fold {}'.format(fold+1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

    model = AttentionModel(
        hparams['n_cnn_layers'],
        hparams['rnn_dim'],
        hparams['h_rnn_layers'],
        hparams['n_rnn_layers'],
        hparams['n_class'],
        hparams['stride'],
        hparams['dropout']
    ).to(device).float()
    optimizer, scheduler = optimizerNet(model, hparams)

    train2(model, device, train_loader, criterion, optimizer, scheduler, epochs, iter_meter, experiment)
    test2(model, device, test_loader, criterion, optimizer, scheduler, epochs, iter_meter, experiment)

Fold 1
Train Epoch: 0 	Loss: 1.3231	Accuracy: 0.4094
Train Epoch: 1 	Loss: 1.2943	Accuracy: 0.4208
Train Epoch: 2 	Loss: 1.2612	Accuracy: 0.4254
Train Epoch: 3 	Loss: 1.2361	Accuracy: 0.4287
Train Epoch: 4 	Loss: 1.2181	Accuracy: 0.4314
Train Epoch: 5 	Loss: 1.2042	Accuracy: 0.4322
Train Epoch: 6 	Loss: 1.1933	Accuracy: 0.4343
Train Epoch: 7 	Loss: 1.1842	Accuracy: 0.4353
Train Epoch: 8 	Loss: 1.1768	Accuracy: 0.4363
Train Epoch: 9 	Loss: 1.1705	Accuracy: 0.4373
Train Epoch: 10 	Loss: 1.1648	Accuracy: 0.4383
Train Epoch: 11 	Loss: 1.1597	Accuracy: 0.4389
Train Epoch: 12 	Loss: 1.1551	Accuracy: 0.4401
Train Epoch: 13 	Loss: 1.1508	Accuracy: 0.4414
Train Epoch: 14 	Loss: 1.1470	Accuracy: 0.4426
Train Epoch: 15 	Loss: 1.1436	Accuracy: 0.4438
Train Epoch: 16 	Loss: 1.1404	Accuracy: 0.4450
Train Epoch: 17 	Loss: 1.1374	Accuracy: 0.4464
Train Epoch: 18 	Loss: 1.1343	Accuracy: 0.4481
Train Epoch: 19 	Loss: 1.1316	Accuracy: 0.4494
Train Epoch: 20 	Loss: 1.1289	Accuracy: 0.4506
Train Epoch: 21 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform_(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.3563	Accuracy: 0.3432
Train Epoch: 1 	Loss: 1.3381	Accuracy: 0.3544
Train Epoch: 2 	Loss: 1.3018	Accuracy: 0.3771
Train Epoch: 3 	Loss: 1.2693	Accuracy: 0.3915
Train Epoch: 4 	Loss: 1.2453	Accuracy: 0.4016
Train Epoch: 5 	Loss: 1.2273	Accuracy: 0.4078
Train Epoch: 6 	Loss: 1.2135	Accuracy: 0.4128
Train Epoch: 7 	Loss: 1.2025	Accuracy: 0.4171
Train Epoch: 8 	Loss: 1.1933	Accuracy: 0.4205
Train Epoch: 9 	Loss: 1.1855	Accuracy: 0.4237
Train Epoch: 10 	Loss: 1.1788	Accuracy: 0.4258
Train Epoch: 11 	Loss: 1.1728	Accuracy: 0.4281
Train Epoch: 12 	Loss: 1.1673	Accuracy: 0.4304
Train Epoch: 13 	Loss: 1.1622	Accuracy: 0.4323
Train Epoch: 14 	Loss: 1.1579	Accuracy: 0.4340
Train Epoch: 15 	Loss: 1.1538	Accuracy: 0.4354
Train Epoch: 16 	Loss: 1.1503	Accuracy: 0.4368
Train Epoch: 17 	Loss: 1.1468	Accuracy: 0.4385
Train Epoch: 18 	Loss: 1.1437	Accuracy: 0.4402
Train Epoch: 19 	Loss: 1.1407	Accuracy: 0.4415
Train Epoch: 20 	Loss: 1.1380	Accuracy: 0.4429
Train Epoch: 21 	Loss: 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform_(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.3350	Accuracy: 0.3808
Train Epoch: 1 	Loss: 1.3169	Accuracy: 0.4012
Train Epoch: 2 	Loss: 1.2993	Accuracy: 0.4145
Train Epoch: 3 	Loss: 1.2847	Accuracy: 0.4203
Train Epoch: 4 	Loss: 1.2630	Accuracy: 0.4227
Train Epoch: 5 	Loss: 1.2450	Accuracy: 0.4253
Train Epoch: 6 	Loss: 1.2309	Accuracy: 0.4264
Train Epoch: 7 	Loss: 1.2190	Accuracy: 0.4274
Train Epoch: 8 	Loss: 1.2089	Accuracy: 0.4289
Train Epoch: 9 	Loss: 1.2000	Accuracy: 0.4298
Train Epoch: 10 	Loss: 1.1925	Accuracy: 0.4313
Train Epoch: 11 	Loss: 1.1859	Accuracy: 0.4327
Train Epoch: 12 	Loss: 1.1800	Accuracy: 0.4337
Train Epoch: 13 	Loss: 1.1748	Accuracy: 0.4350
Train Epoch: 14 	Loss: 1.1699	Accuracy: 0.4364
Train Epoch: 15 	Loss: 1.1654	Accuracy: 0.4379
Train Epoch: 16 	Loss: 1.1615	Accuracy: 0.4391
Train Epoch: 17 	Loss: 1.1579	Accuracy: 0.4400
Train Epoch: 18 	Loss: 1.1545	Accuracy: 0.4413
Train Epoch: 19 	Loss: 1.1512	Accuracy: 0.4425
Train Epoch: 20 	Loss: 1.1482	Accuracy: 0.4435
Train Epoch: 21 	Loss: 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform_(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.3600	Accuracy: 0.2730
Train Epoch: 1 	Loss: 1.3029	Accuracy: 0.3483
Train Epoch: 2 	Loss: 1.2600	Accuracy: 0.3773
Train Epoch: 3 	Loss: 1.2321	Accuracy: 0.3922
Train Epoch: 4 	Loss: 1.2139	Accuracy: 0.4005
Train Epoch: 5 	Loss: 1.1999	Accuracy: 0.4067
Train Epoch: 6 	Loss: 1.1890	Accuracy: 0.4114
Train Epoch: 7 	Loss: 1.1796	Accuracy: 0.4156
Train Epoch: 8 	Loss: 1.1720	Accuracy: 0.4191
Train Epoch: 9 	Loss: 1.1653	Accuracy: 0.4221
Train Epoch: 10 	Loss: 1.1597	Accuracy: 0.4247
Train Epoch: 11 	Loss: 1.1549	Accuracy: 0.4268
Train Epoch: 12 	Loss: 1.1504	Accuracy: 0.4287
Train Epoch: 13 	Loss: 1.1466	Accuracy: 0.4307
Train Epoch: 14 	Loss: 1.1432	Accuracy: 0.4327
Train Epoch: 15 	Loss: 1.1401	Accuracy: 0.4341
Train Epoch: 16 	Loss: 1.1374	Accuracy: 0.4356
Train Epoch: 17 	Loss: 1.1348	Accuracy: 0.4370
Train Epoch: 18 	Loss: 1.1321	Accuracy: 0.4387
Train Epoch: 19 	Loss: 1.1296	Accuracy: 0.4402
Train Epoch: 20 	Loss: 1.1273	Accuracy: 0.4417
Train Epoch: 21 	Loss: 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform_(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.2992	Accuracy: 0.3976
Train Epoch: 1 	Loss: 1.2504	Accuracy: 0.4140
Train Epoch: 2 	Loss: 1.2204	Accuracy: 0.4215
Train Epoch: 3 	Loss: 1.2006	Accuracy: 0.4264
Train Epoch: 4 	Loss: 1.1872	Accuracy: 0.4296
Train Epoch: 5 	Loss: 1.1768	Accuracy: 0.4319
Train Epoch: 6 	Loss: 1.1682	Accuracy: 0.4341
Train Epoch: 7 	Loss: 1.1616	Accuracy: 0.4356
Train Epoch: 8 	Loss: 1.1560	Accuracy: 0.4372
Train Epoch: 9 	Loss: 1.1511	Accuracy: 0.4387
Train Epoch: 10 	Loss: 1.1469	Accuracy: 0.4399
Train Epoch: 11 	Loss: 1.1431	Accuracy: 0.4411
Train Epoch: 12 	Loss: 1.1396	Accuracy: 0.4428
Train Epoch: 13 	Loss: 1.1364	Accuracy: 0.4440
Train Epoch: 14 	Loss: 1.1334	Accuracy: 0.4451
Train Epoch: 15 	Loss: 1.1307	Accuracy: 0.4464
Train Epoch: 16 	Loss: 1.1283	Accuracy: 0.4473
Train Epoch: 17 	Loss: 1.1259	Accuracy: 0.4486
Train Epoch: 18 	Loss: 1.1238	Accuracy: 0.4495
Train Epoch: 19 	Loss: 1.1217	Accuracy: 0.4505
Train Epoch: 20 	Loss: 1.1198	Accuracy: 0.4514
Train Epoch: 21 	Loss: 

## MODEL WITHOUT K-FOLDS

As all the results seems to be useful, I'm going to train a random model with more epochs to see when it converges.

In [2]:
train_data, test_data = dataLoaderPickle('Scripted_Eng_GAD7')

Training size: 10772
Testing size: 2693


In [3]:
#Constants
learning_Rate = 0.0005
batch_size = 128
epochs = 150
experiment = Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 6,
    "n_rnn_layers": 1,
    "rnn_dim": 256,
    "h_rnn_layers": 128,
    "n_class": 4,
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


In [5]:
#Building and training Model
model = AttentionModel(
    hparams['n_cnn_layers'],
    hparams['rnn_dim'],
    hparams['h_rnn_layers'],
    hparams['n_rnn_layers'],
    hparams['n_class'],
    hparams['stride'],
    hparams['dropout']
).to(device).float()

print(model)
print('Number Model Parameters', sum([param.nelement() for param in model.parameters()]))

#Optimizing Model
optimizer, scheduler = optimizerNet(model, hparams)
weights = [1.0,1.0,1.0,0.85]
class_weights = torch.FloatTensor(weights)
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
iter_meter = IterMeter()

train2(model, device, train_data, criterion, optimizer, scheduler, epochs, iter_meter, experiment)
test2(model, device, test_data, criterion, optimizer, scheduler, epochs, iter_meter, experiment)

AttentionModel(
  (dense): Conv1d(500, 32, kernel_size=(1,), stride=(1,))
  (cnn): Sequential(
    (0): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (2): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_runn